In [2]:
import numpy as np 
import pandas as pd

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu
data = pd.read_csv('/kaggle/input/ner-data/ner_datasetreference.csv', encoding='latin1')

# Điền các giá trị thiếu trong cột 'Sentence #' bằng phương pháp 'forward fill'
data['Sentence #'].fillna(method='ffill', inplace=True)

# Chia dữ liệu theo số thứ tự câu
sentences = data['Sentence #'].unique()
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

# Lấy các câu tương ứng trong dữ liệu gốc
train_data = data[data['Sentence #'].isin(train_sentences)]
test_data = data[data['Sentence #'].isin(test_sentences)]

# Lưu dữ liệu huấn luyện
train_data.to_csv('/kaggle/working/train.csv', index=False)

# Lưu dữ liệu kiểm tra
test_data.to_csv('/kaggle/working/test.csv', index=False)

print("Dữ liệu đã được chia thành các tệp train.csv và test.csv")



/tmp/ipykernel_33/1962503247.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Sentence #'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_33/1962503247.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Sentence #'].fillna(method='ffill', inplace=True)


Dữ liệu đã được chia thành các tệp train.csv và test.csv


In [84]:
file_path = "/kaggle/working/test.csv"
df = pd.read_csv(file_path)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-per\b', value='PERSON', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-per\b', value='PERSON', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-org\b', value='ORG', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-org\b', value='ORG', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-gpe\b', value='GPE', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-gpe\b', value='GPE', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-art\b', value='ART', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-art\b', value='ART', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-eve\b', value='EVE', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-eve\b', value='EVE', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-nat\b', value='NAT', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-nat\b', value='NAT', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-geo\b', value='GEO', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-geo\b', value='GEO', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bB-tim\b', value='TIM', regex=True)
df['Tag'] = df['Tag'].replace(to_replace=r'\bI-tim\b', value='TIM', regex=True)
value_counts = df['Tag'].value_counts()
print("Số lượng và các giá trị duy nhất trong cột 'Tag' sau khi thay đổi:")
for value, count in value_counts.items():
    print(f"{value}: {count}")


Số lượng và các giá trị duy nhất trong cột 'Tag' sau khi thay đổi:
O: 176877
GEO: 9114
ORG: 7228
PERSON: 6834
TIM: 5349
GPE: 3214
ART: 144
EVE: 113
NAT: 62


In [85]:
# Nhóm các từ và nhãn theo số thứ tự của câu
grouped = df.groupby('Sentence #').apply(lambda x: [(word, tag) for word, tag in zip(x['Word'].values, x['Tag'].values)]).reset_index(name='Sentence')


/tmp/ipykernel_33/80755995.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby('Sentence #').apply(lambda x: [(word, tag) for word, tag in zip(x['Word'].values, x['Tag'].values)]).reset_index(name='Sentence')


In [86]:
# Hàm chuyển đổi các từ và nhãn thành định dạng "word<tag>"
def format_sentence(sentence):
    return ' '.join([f"{word}<{tag}>" for word, tag in sentence])

# Áp dụng hàm để gộp các từ và nhãn thành câu
grouped['Formatted Sentence'] = grouped['Sentence'].apply(format_sentence)

# Hiển thị câu đầu tiên
print(grouped['Formatted Sentence'].iloc[0])
grouped['Formatted Sentence'].to_csv('testner.csv')

Lebanon<GEO> has<O> suffered<O> a<O> series<O> of<O> bombings<O> since<O> the<O> massive<O> explosion<O> in<O> February<TIM> that<O> killed<O> former<O> Prime<PERSON> Minister<O> Rafik<PERSON> Hariri<PERSON> and<O> 20<O> other<O> people<O> .<O>


In [87]:
import re

input_file = "/kaggle/working/testner.csv"
output_file = "output.csv"

# Đọc nội dung từ file input vào biến
with open(input_file, 'r', encoding='utf-8') as file:
    content = file.read()

# Xử lý nội dung để xóa các ký tự < và >
cleaned_content = re.sub(r'<[^>]*>', '', content)

# Ghi nội dung đã được xử lý vào file output
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(cleaned_content)

print(f"Đã xử lý và lưu nội dung vào file {output_file}")


Đã xử lý và lưu nội dung vào file output.csv


In [88]:
import spacy

# Tải mô hình ngôn ngữ tiếng Việt của SpaCy
nlp = spacy.load("en_core_web_sm")

text = "He blames Iceland's economic calamity on commercial bankers."
def ner_spacy(text):
# Xử lý văn bản bằng SpaCy
    doc = nlp(text)
    # Chuỗi để lưu kết quả in
    output_string = ""
    # Lặp qua các token trong văn bản
    for token in doc:
        if token.ent_type_:
            output_string += f"{token.text}<{token.ent_type_}> "
        else:
            output_string += f"{token.text}<O> "
    return output_string


In [90]:
file_path = "/kaggle/working/output.csv"
df = pd.read_csv(file_path, usecols=[1])
file_predic = df.iloc[:, 0].tolist()

In [91]:
file_path = "/kaggle/working/testner.csv"
df = pd.read_csv(file_path, usecols=[1])
file_tesst = df.iloc[:, 0].tolist()


['Lebanon<GEO> has<O> suffered<O> a<O> series<O> of<O> bombings<O> since<O> the<O> massive<O> explosion<O> in<O> February<TIM> that<O> killed<O> former<O> Prime<PER> Minister<O> Rafik<PER> Hariri<PER> and<O> 20<O> other<O> people<O> .<O>',
 "The<O> global<O> financial<O> crisis<O> has<O> left<O> Iceland<ORG> 's<O> economy<O> in<O> shambles<O> .<O>",
 'Israeli<GPE> officials<O> say<O> Prime<PER> Minister<PER> Ariel<PER> Sharon<PER> will<O> undergo<O> a<O> medical<O> procedure<O> Thursday<TIM> to<O> close<O> a<O> tiny<O> hole<O> in<O> his<O> heart<O> discovered<O> during<O> treatment<O> for<O> a<O> minor<O> stroke<O> suffered<O> last<O> month<O> .<O>',
 'Doctors<O> say<O> they<O> expect<O> Mr.<PER> Sharon<PER> will<O> make<O> a<O> full<O> recovery<O> .<O>',
 'SpaceShipOne<ART> designer<O> Burt<PER> Rutan<PER> accepted<O> the<O> Ansari<ART> X<ART> Prize<ART> money<O> and<O> a<O> trophy<O> on<O> behalf<O> of<O> his<O> team<O> Saturday<TIM> during<O> an<O> awards<O> ceremony<O> in<O> the<O>

In [92]:
file_tesst

['Lebanon<GEO> has<O> suffered<O> a<O> series<O> of<O> bombings<O> since<O> the<O> massive<O> explosion<O> in<O> February<TIM> that<O> killed<O> former<O> Prime<PER> Minister<O> Rafik<PER> Hariri<PER> and<O> 20<O> other<O> people<O> .<O>',
 "The<O> global<O> financial<O> crisis<O> has<O> left<O> Iceland<ORG> 's<O> economy<O> in<O> shambles<O> .<O>",
 'Israeli<GPE> officials<O> say<O> Prime<PER> Minister<PER> Ariel<PER> Sharon<PER> will<O> undergo<O> a<O> medical<O> procedure<O> Thursday<TIM> to<O> close<O> a<O> tiny<O> hole<O> in<O> his<O> heart<O> discovered<O> during<O> treatment<O> for<O> a<O> minor<O> stroke<O> suffered<O> last<O> month<O> .<O>',
 'Doctors<O> say<O> they<O> expect<O> Mr.<PER> Sharon<PER> will<O> make<O> a<O> full<O> recovery<O> .<O>',
 'SpaceShipOne<ART> designer<O> Burt<PER> Rutan<PER> accepted<O> the<O> Ansari<ART> X<ART> Prize<ART> money<O> and<O> a<O> trophy<O> on<O> behalf<O> of<O> his<O> team<O> Saturday<TIM> during<O> an<O> awards<O> ceremony<O> in<O> the<O>

In [93]:
# Load model ngôn ngữ tiếng Anh của SpaCy
nlp = spacy.load("en_core_web_sm")

# Danh sách lưu trữ kết quả gán nhãn thực thể cho từng câu của file 1
predic = []

# Gán nhãn thực thể cho từng câu trong file 1
for sentence in file_predic:
    predic.append(ner_spacy(sentence))


In [94]:
predic

['Lebanon<GPE> has<O> suffered<O> a<O> series<O> of<O> bombings<O> since<O> the<O> massive<O> explosion<O> in<O> February<DATE> that<O> killed<O> former<O> Prime<O> Minister<O> Rafik<PERSON> Hariri<PERSON> and<O> 20<CARDINAL> other<O> people<O> .<O> ',
 "The<O> global<O> financial<O> crisis<O> has<O> left<O> Iceland<GPE> 's<O> economy<O> in<O> shambles<O> .<O> ",
 'Israeli<NORP> officials<O> say<O> Prime<O> Minister<O> Ariel<PERSON> Sharon<PERSON> will<O> undergo<O> a<O> medical<O> procedure<O> Thursday<DATE> to<O> close<O> a<O> tiny<O> hole<O> in<O> his<O> heart<O> discovered<O> during<O> treatment<O> for<O> a<O> minor<O> stroke<O> suffered<O> last<DATE> month<DATE> .<O> ',
 'Doctors<O> say<O> they<O> expect<O> Mr.<O> Sharon<PERSON> will<O> make<O> a<O> full<O> recovery<O> .<O> ',
 'SpaceShipOne<ORG> designer<O> Burt<PERSON> Rutan<PERSON> accepted<O> the<O> Ansari<O> X<O> Prize<O> money<O> and<O> a<O> trophy<O> on<O> behalf<O> of<O> his<O> team<O> Saturday<DATE> during<O> an<O> awards

In [96]:
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def extract_words_and_tags(sentence):
    words_and_tags = re.findall(r'(\S+)<(.*?)>', sentence)
    return words_and_tags

# Hàm để so sánh các nhãn
def compare_tags(words_and_tags1, words_and_tags2):
    y_true = []
    y_pred = []

    for (word1, tag1), (word2, tag2) in zip(words_and_tags1, words_and_tags2):
        if word1 == word2:
            y_true.append(tag1)
            y_pred.append(tag2)
        else:
            y_true.append('O')
            y_pred.append('O')

    return y_true, y_pred

# Tạo các danh sách để chứa các nhãn thực tế và nhãn dự đoán
all_true_tags = []
all_pred_tags = []

for sent1, sent2 in zip(file_tesst, predic):
    words_and_tags1 = extract_words_and_tags(sent1)
    words_and_tags2 = extract_words_and_tags(sent2)
    
    true_tags, pred_tags = compare_tags(words_and_tags1, words_and_tags2)
    all_true_tags.extend(true_tags)
    all_pred_tags.extend(pred_tags)

# Tính toán các độ đo
accuracy = accuracy_score(all_true_tags, all_pred_tags)
precision = precision_score(all_true_tags, all_pred_tags, average='macro', zero_division=1)
recall = recall_score(all_true_tags, all_pred_tags, average='macro', zero_division=1)
f1 = f1_score(all_true_tags, all_pred_tags, average='macro', zero_division=1)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Accuracy: 0.83
Precision: 0.31
Recall: 0.71
F1-Score: 0.07
